In [1]:
from IPython import display
display.clear_output()
 
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.216 🚀 Python-3.10.12 torch-2.1.1+cu121 CUDA:0 (NVIDIA H100 80GB HBM3, 81008MiB)
Setup complete ✅ (16 CPUs, 221.2 GB RAM, 48.1/193.6 GB disk)


In [2]:
import torch
torch.cuda.is_available()

True

In [7]:
!nvidia-smi

Fri Nov 24 15:51:04 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.12             Driver Version: 535.104.12   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA H100 80GB HBM3          On  | 00000000:00:05.0 Off |                    0 |
| N/A   35C    P0              74W / 700W |      7MiB / 81559MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [3]:
!conda list

# packages in environment at /home/ubuntu/miniforge:
#
# Name                    Version                   Build  Channel
_libgcc_mutex             0.1                 conda_forge    conda-forge
_openmp_mutex             4.5                       2_gnu    conda-forge
anyio                     4.1.0                    pypi_0    pypi
argon2-cffi               23.1.0                   pypi_0    pypi
argon2-cffi-bindings      21.2.0                   pypi_0    pypi
arrow                     1.3.0                    pypi_0    pypi
asttokens                 2.4.1                    pypi_0    pypi
async-lru                 2.0.4                    pypi_0    pypi
attrs                     23.1.0                   pypi_0    pypi
babel                     2.13.1                   pypi_0    pypi
beautifulsoup4            4.12.2                   pypi_0    pypi
bleach                    6.1.0                    pypi_0    pypi
boltons                   23.0.0             pyhd8ed1ab_0    conda-forge

In [8]:
import pandas as pd
import glob
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.preprocessing import StandardScaler

In [5]:
# Keypoints의 상대좌표로 변환하는 함수
def box_nae_sangdae_jwapyo(bbox, keypoints):
    keypoints = keypoints.copy().astype(float)
    x0, y0, x1, y1 = bbox[:4].astype(float)
    keypoints[:, 0] = (keypoints[:, 0] - x0) / (x1 - x0)
    keypoints[:, 1] = (keypoints[:, 1] - y0) / (y1 - y0)
    return keypoints

print("데이터 불러오기 시작...")

# 데이터 불러오기 및 전처리
path = './csv_data'
all_files = glob.glob(path + "/*.csv")
li = []

for filename in all_files:
    with open(filename, 'r') as temp_f:
        col_count = len(temp_f.readline().split(','))
    column_names = list(range(col_count))
    df = pd.read_csv(filename, index_col=None, names=column_names)
    li.append(df)

data = pd.concat(li, axis=0, ignore_index=True)

print("데이터 정규화 진행 중...")

# Keypoints 데이터 정규화
for i in range(len(data)):
    bbox = data.iloc[i, 0:4].values  # bbox 정보
    keypoints = data.iloc[i, 6:40].values.reshape(-1, 2)  # keypoints 정보
    normalized_kpts = box_nae_sangdae_jwapyo(bbox, keypoints)
    data.iloc[i, 6:40] = normalized_kpts.flatten()

# inf 값 처리
X = np.where(np.isinf(data.iloc[:, 6:40].values), np.nan, data.iloc[:, 6:40].values)

# NaN 값을 평균으로 대체
mean_value = np.nanmean(X, axis=0)
X = np.where(np.isnan(X), mean_value, X)

# 데이터 정규화
scaler = StandardScaler()
X = scaler.fit_transform(X)

print("데이터 분할 및 라벨 인코딩...")

# 라벨 인코딩
y_encoded, _ = pd.factorize(data.iloc[:, -1])

# 데이터 분할: 학습 데이터와 테스트 데이터
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

print("모델 학습 시작...")

# XGBoost 모델 생성 및 학습
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
model.fit(X_train, y_train)

print("모델 학습 완료. 성능 평가 진행 중...")

# 테스트 데이터에 대한 예측 수행 및 성능 평가
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')
conf_matrix = confusion_matrix(y_test, y_pred)

print("모델 성능 평가 결과:")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Confusion Matrix:\n", conf_matrix)

데이터 불러오기 시작...
데이터 정규화 진행 중...


/tmp/ipykernel_234894/2903844737.py:14: RuntimeWarning: divide by zero encountered in divide
  keypoints[:, 1] = (keypoints[:, 1] - y0) / (y1 - y0)
/tmp/ipykernel_234894/2903844737.py:13: RuntimeWarning: divide by zero encountered in divide
  keypoints[:, 0] = (keypoints[:, 0] - x0) / (x1 - x0)


KeyboardInterrupt: 

In [7]:
# read csv file
df = pd.read_csv('./csv_data/C012_A02_SY04_P01_S01_02DAS_results.csv', index_col=None, names=column_names)
df.head()
# 0: 프레임번호
# 1: 객체 번호
# 2~5: 객체 바운딩박스 x,y좌표
# 6~22: X 좌표 17개
# 23~39: y 좌표 17개
# 40~56: V 값 17개 (keypoint 검출 정확도)
# 57: action 값

,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,57
0,4,1,1077,303,1289,966,1236.3323,1218.0741,1234.3853,1155.8589,...,0.989144,0.239062,0.900393,0.996721,0.999366,0.987731,0.997305,0.966473,0.987615,normal
1,4,0,1298,465,1501,1038,1404.1134,1398.7839,1417.0029,1387.2267,...,0.968185,0.720749,0.864559,0.997265,0.998310,0.987325,0.991492,0.944912,0.957058,normal
2,5,1,1077,303,1289,966,1236.1199,1217.9410,1234.1296,1155.9858,...,0.989109,0.248220,0.901391,0.996771,0.999363,0.987848,0.997277,0.966429,0.987413,normal
3,5,0,1297,463,1501,1037,1406.3157,1404.2374,1417.4060,1396.5735,...,0.933354,0.815358,0.792658,0.994343,0.994084,0.977477,0.974426,0.924713,0.915486,normal
4,6,1,1077,303,1289,965,1236.3723,1218.2216,1234.1138,1156.5172,...,0.989405,0.258105,0.906296,0.996754,0.999360,0.987884,0.997284,0.966410,0.987398,normal


In [9]:
# 바운딩 박스 로직처리 (바운딩 박스가 클시 성인(1), 작을시 아동(0))
# 객체 번호를 기준으로 한명의 사람
bbox = df[df[1] == 0].iloc[0, 2:6].values
bbox
# 객체 번호를 기준으로 한명의 사람의 좌표
# 사람들중 가장 큰사람을 성인이라 가정


array([1298, 465, 1501, 1038], dtype=object)